In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dota-2-matches/players.csv
/kaggle/input/dota-2-matches/ability_upgrades.csv
/kaggle/input/dota-2-matches/test_player.csv
/kaggle/input/dota-2-matches/match_outcomes.csv
/kaggle/input/dota-2-matches/teamfights_players.csv
/kaggle/input/dota-2-matches/chat.csv
/kaggle/input/dota-2-matches/purchase_log.csv
/kaggle/input/dota-2-matches/test_labels.csv
/kaggle/input/dota-2-matches/ability_ids.csv
/kaggle/input/dota-2-matches/item_ids.csv
/kaggle/input/dota-2-matches/player_time.csv
/kaggle/input/dota-2-matches/patch_dates.csv
/kaggle/input/dota-2-matches/yasp_sample.json
/kaggle/input/dota-2-matches/match.csv
/kaggle/input/dota-2-matches/player_ratings.csv
/kaggle/input/dota-2-matches/hero_names.csv
/kaggle/input/dota-2-matches/cluster_regions.csv
/kaggle/input/dota-2-matches/teamfights.csv
/kaggle/input/dota-2-matches/objectives.csv


The first question I want to answer is: Can we predict the match outcome based on the aggregates of heroes in the match?

To start we are going to read in the data, and start to combine the CSVs to get information about a single match. 

In [2]:
#Each match has one entry, in this case we are focused on match_id, and radiant_win only as our target.
columnsWeWant = ['match_id','radiant_win']
match = pd.read_csv('/kaggle/input/dota-2-matches/match.csv')
match = match[columnsWeWant]
match.head(10)

,match_id,radiant_win
0,0,True
1,1,False
2,2,False
3,3,False
4,4,True
5,5,True
6,6,True
7,7,True
8,8,False
9,9,False


In [3]:
#We are only intrested in this case on the hero_id and match_id, and player_slot. Note that: Player_slot: 0-4 are Radiant, 128-132 are Dire
columnsWeWant = ['match_id','hero_id','player_slot']
players = pd.read_csv('/kaggle/input/dota-2-matches/players.csv')
players = players[columnsWeWant]
players.head(10)

,match_id,hero_id,player_slot
0,0,86,0
1,0,51,1
2,0,83,2
3,0,11,3
4,0,67,4
5,0,106,128
6,0,102,129
7,0,46,130
8,0,7,131
9,0,73,132


In [4]:
#We now merge the two tables above to get all 10 hero_id for each match_id, we also note which of these heroes lost and won. 
merged = players.merge(match)
merged

,match_id,hero_id,player_slot,radiant_win
0,0,86,0,True
1,0,51,1,True
2,0,83,2,True
3,0,11,3,True
4,0,67,4,True
...,...,...,...,...
499995,49999,100,128,False
499996,49999,9,129,False
499997,49999,90,130,False
499998,49999,73,131,False


In [5]:
#This groups the match_id and make new columns for each hero_id
byMatch = merged.groupby('match_id').hero_id.unique()
byMatch = pd.DataFrame(byMatch.values.tolist(), index=byMatch.index).reset_index()
byMatch

,match_id,0,1,2,3,4,5,6,7,8,9
0,0,86,51,83,11,67,106,102,46,7,73.0
1,1,7,82,71,39,21,73,22,5,67,106.0
2,2,51,109,9,41,27,38,7,10,12,85.0
3,3,50,44,32,26,39,78,19,31,40,47.0
4,4,8,39,55,87,69,101,100,22,67,21.0
...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,73,86,21,20,14,32,7,109,35,112.0
49996,49996,93,74,100,32,85,36,1,112,60,71.0
49997,49997,100,68,75,39,44,28,102,21,9,23.0
49998,49998,56,50,2,72,30,46,7,29,44,3.0


In [6]:
#Here we make a new column by using the player_slot compared to radiant_win to see if they won or lost the match.
merged["player_won"] = ( (merged['radiant_win'] & (merged['player_slot'] < 5)) | ((merged['radiant_win']==False) & (merged["player_slot"] > 5)) )
merged.head(20)

,match_id,hero_id,player_slot,radiant_win,player_won
0,0,86,0,True,True
1,0,51,1,True,True
2,0,83,2,True,True
3,0,11,3,True,True
4,0,67,4,True,True
5,0,106,128,True,False
6,0,102,129,True,False
7,0,46,130,True,False
8,0,7,131,True,False
9,0,73,132,True,False
